In [1]:
import mysql.connector as con

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [2]:
fd = open('SQLData/claims_v1_live.sql', 'r')
sqlFile = fd.readlines()
fd.close()

In [3]:

commands = []
statement = ""

for line in tqdm(sqlFile):

    line = line.replace('\n','')
    
    if re.search(r'--', line):  # ignore sql comment lines
        continue
       
    if re.search(r'/*!',line):
        continue
    
    if not re.search(r';$', line):  # keep appending lines that don't end in ';'
        statement = statement + line
        
    else:  # when you get a line ending in ';' then exec statement and reset for next statement
        statement = statement + line
        commands.append(statement)

        statement = ""



100%|██████████| 1026688/1026688 [00:06<00:00, 150130.24it/s]


In [4]:
print(commands[5])

INSERT INTO `cares` (`id`, `name`, `base_name`, `type`, `active`, `created_at`, `updated_at`, `nlp_result`, `type_id`, `meta`, `drug_generic_id`, `parent_id`, `cve_version`) VALUES(729, 'DOCITAXEL 80MG INJ - 1 VIAL', NULL, NULL, 1, '2018-02-14 12:32:17', '2018-02-14 12:32:17', NULL, NULL, NULL, NULL, NULL, NULL),(730, 'DOPAMINE INJ 40MG - 1 AMPOULE', NULL, NULL, 1, '2018-02-14 12:32:17', '2018-02-14 12:32:17', NULL, NULL, NULL, NULL, NULL, NULL),(731, 'DRUJELA GEL 10G - 1 TUBE', NULL, NULL, 1, '2018-02-14 12:32:17', '2018-02-14 12:32:17', NULL, NULL, NULL, NULL, NULL, NULL),(732, 'DUCOLAX SUPPOSITORY - 1 SUPPO', NULL, NULL, 1, '2018-02-14 12:32:17', '2018-02-14 12:32:17', NULL, NULL, NULL, NULL, NULL, NULL),(733, 'DUOVIR 650MG TAB - 1 TABLET', NULL, NULL, 1, '2018-02-14 12:32:17', '2018-02-14 12:32:17', NULL, NULL, NULL, NULL, NULL, NULL),(734, 'DUPHASTON TABLET 10MG - 1 TABLET', NULL, NULL, 1, '2018-02-14 12:32:17', '2018-02-14 12:32:17', NULL, NULL, NULL, NULL, NULL, NULL),(735, 'DYN

In [5]:
len(commands)

2430

In [6]:
cur = con.connect(host='localhost',database='mysql',user='root',password='52dna52myaim')

In [8]:
cursor =cur.cursor()

In [111]:


skip = []

cursor.execute("drop table cares;")
cursor.execute("drop table care_types;")
cursor.execute("drop table claims;")
for command in tqdm(commands):

    try:
        cursor.execute(command)
    except OperationalError:
        skip.append(command)


100%|█████████▉| 2424/2430 [01:02<00:27,  4.55s/it] 

DatabaseError: 1824 (HY000): Failed to open the referenced table 'drug_generics'

In [112]:
len(skip)

0

In [9]:
cursor.execute("select * from cares;")

In [10]:
cares = cursor.fetchall()

In [11]:
len(cares)

83124

In [13]:
for n in tqdm(cares[0:5]):
    print(n)

100%|██████████| 5/5 [00:00<00:00, 2088.80it/s]

(2, 'Chest X-ray', None, 'INVESTIGATION', 1, datetime.datetime(2019, 4, 23, 15, 27, 50), datetime.datetime(2019, 9, 26, 18, 40, 31), None, None, None, None, None, None)
(6, 'Gentamycin', None, 'DRUG', 1, datetime.datetime(2019, 4, 23, 15, 27, 50), datetime.datetime(2019, 10, 5, 18, 14, 30), None, 1, None, 927, None, None)
(7, 'Amikacin', None, 'DRUG', 1, datetime.datetime(2019, 4, 23, 15, 27, 50), datetime.datetime(2019, 10, 6, 1, 58, 26), None, 1, None, 178, None, None)
(8, 'Metronidazole 500mg', None, 'DRUG', 1, datetime.datetime(2019, 4, 23, 15, 27, 50), datetime.datetime(2019, 4, 23, 15, 27, 50), None, None, None, None, None, None)
(9, 'Consultation', None, None, 1, datetime.datetime(2019, 4, 23, 15, 27, 50), datetime.datetime(2019, 4, 23, 15, 27, 50), None, None, None, None, None, None)


In [ ]:
"""
tables: 

cares, care_types,claim_comments,claim_diagnosis,claim_items,claims,comments,
diagnosis, diagnosis_categories, drug_generics, enrollees, hmos, hmos_piles, providers, provider_tariffs

"""

In [14]:
cursor.execute("show columns from cares;")
cares_columns = cursor.fetchall()
cares_columns

[('id', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('name', 'varchar(255)', 'NO', 'MUL', None, ''),
 ('base_name', 'varchar(255)', 'YES', 'UNI', None, ''),
 ('type', 'varchar(20)', 'YES', 'MUL', None, ''),
 ('active', 'tinyint(1)', 'NO', '', '1', ''),
 ('created_at', 'datetime', 'NO', '', None, ''),
 ('updated_at', 'datetime', 'YES', '', None, ''),
 ('nlp_result', 'longtext', 'YES', '', None, ''),
 ('type_id', 'int', 'YES', 'MUL', None, ''),
 ('meta', 'longtext', 'YES', '', None, ''),
 ('drug_generic_id', 'int', 'YES', 'MUL', None, ''),
 ('parent_id', 'int', 'YES', 'MUL', None, ''),
 ('cve_version', 'int', 'YES', '', None, '')]

In [17]:
tables = ['cares', 'care_types', 'claim_comments', 'claim_diagnosis', 'claim_items', 'claims,comments',\
          'diagnosis', 'diagnosis_categories', 'drug_generics', 'enrollees', 'hmos', 'hmos_piles', \
          'providers', 'provider_tariffs']

In [ ]:
for 
cursor.execute("select * from care_types;")
care_types = cursor.fetchall()

cursor.execute("select * from care_types;")
care_types = cursor.fetchall()
